pip install selenium

### Função pra transformar a lista de infos na string a se renomear os arquivos

In [ ]:
def formatar_string(list_info):
    
    meta = list_info[3][17:20]
    município = list_info[1]
    cliente = list_info[6].split()
    data = list_info[7].split('/')
    visita = list_info[4]
    
    if(meta == "2.3"):
        nome_do_arquivo = meta + "_" + município + "_" + cliente[0] + " " + cliente[-1] + "_" + data[2] + "." + data[1] + "." + data[0] + "_" + visita + ".pdf"
    else:
        nome_do_arquivo = meta + "_" + município + "_" + cliente[0] + " " + cliente[1] + " et al" + "_" + data[2] + "." + data[1] + "." + data[0] + ".pdf"
            
    return nome_do_arquivo

### Função pra pegar o nome original do arquivo

In [ ]:
def pegar_nome_original(timer, directory):
    
    time.sleep(timer)
    """
        Tem formas avançadas de determinar se o arquivo foi baixado. Aqui alguns links
        https://stackoverflow.com/questions/34338897/python-selenium-find-out-when-a-download-has-completed/34338926#34338926
        https://stackoverflow.com/questions/21746750/check-and-wait-until-a-file-exists-to-read-it
    """
    
    data_criacao = lambda f: f.stat().st_ctime

    files = directory.glob('*.pdf')
    nome_original = sorted(files, key=data_criacao, reverse=True)
    
    """
    Essa função vai pegar todos os pdfs da pasta e colocar o caminho deles em uma lista(nome_original) da qual eu vou pegar o
    primeiro termo que seria o mais recente criado. Então por tudo que é mais sagrado, que não tenha outros arquivos pdfs de
    outra natureza nessa pasta senão vai renomear e mover ele.
    """
    
    if not nome_original:
        nome_original.append("Alguma coisa")

    return nome_original[0] #Tem o endereço completo ou a expressão "Alguma coisa"
    
    

Criando sufixos (n) para arquivos de mesmo nome e talvez assuntos diferentes

In [ ]:
def nome_formatado_sufixo(nome_formatado, n):
    nome_formatado = nome_formatado[:-4]
    nome_formatado = nome_formatado + f" ({n})" + ".pdf"
    
    return nome_formatado
    

### Renomeando e movendo o arquivo baixado

In [ ]:
def renomear_e_mover_arquivo(nome_formatado, nome_original, SRE, path, uregi, diretório_de_destino):

    meta = nome_formatado[:3]
    
    if(nome_original != "Alguma coisa"):
        n = 1
        
        os.rename(nome_original, nome_formatado)
        
        endereco_do_arquivo_a_ser_movido = path + nome_formatado
        
        
        arquivo_de_destino = diretório_de_destino + "\\" + nome_formatado
        my_file = Path(arquivo_de_destino)
        if my_file.is_file():
            
            while my_file.is_file():                
                
                if n == 1:
                    os.rename(nome_formatado, nome_formatado_sufixo(nome_formatado, n))
                else:
                    #nome_formatado_sufixo_menos1 = nome_formatado + f" ({n-1})"
                    os.rename(nome_formatado_sufixo(nome_formatado, n-1), nome_formatado_sufixo(nome_formatado, n))

                endereco_do_arquivo_a_ser_movido = path + nome_formatado_sufixo(nome_formatado, n)
                
                arquivo_de_destino = diretório_de_destino + "\\" + nome_formatado_sufixo(nome_formatado, n)
                my_file = Path(arquivo_de_destino)
                
                if my_file.is_file():
                    n += 1
                else:
                    shutil.move(endereco_do_arquivo_a_ser_movido,diretório_de_destino)
                    break
                    
            with open(fr'{path}\\{uregi[12:]}.txt', 'a') as file:
                file.write(f"O arquivo já existia na pasta mas foi adicionado este com o sufixo ({n})")
                file.write("\n\n\n")
            
        else:
            shutil.move(endereco_do_arquivo_a_ser_movido,diretório_de_destino)
            
    else:
        with open(fr'{path}\\{uregi[12:]}.txt', 'a') as file:
            file.write("O arquivo o arquivo não foi encontrado ou baixado ou renomeado ou movido. Precisa-se de tratamento manual :/")
            file.write("\n\n\n")
        
        

### Gerando a lista que irá para o txt

In [ ]:
def gera_lista_txt(list_info):
    
    meta = list_info[3][17:20]
    lista_para_txt = list_info.copy()
    
    lista_de_nomes = lista_para_txt[6].split()
    lista_de_nomes = lista_de_nomes[:2]
    
    if(meta != "2.3"):
        lista_de_nomes.append("et al")
        
    nomes_em_string = ' '.join(lista_de_nomes)
    
    lista_para_txt[6] = nomes_em_string
        
    return lista_para_txt

### Gerando o arquivo de txt com as infos de cada Uregi

In [ ]:
def arquivo_txt(infos, nome_do_arquivo, uregi, path):

    # Convertendo a lista em string
    infos_str = '; '.join(infos)
    
    with open(fr'{path}\\{uregi[12:]}.txt', 'a') as file:
        file.write(infos_str)
        file.write("\n")
        file.write(nome_do_arquivo)
        file.write("\n\n")

### Executa a ação

In [ ]:
def executa_ação(linha, uregi, timer, directory, path, meta_analisada, diretório_de_destino, linha_limite, original_window, driver, list_info):
    # Clicando na imagem para abrir a aba de download
    driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]/td[10]/a/img").click()
    global scroll
    
    try:
        driver.switch_to.window(driver.window_handles[1])

    except:
        scroll += driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]").rect['height']
        driver.execute_script(f"window.scrollTo(500,{scroll})") #x,y

        linha = linha + 1
        list_info.clear()

    else:
        driver.find_element(By.LINK_TEXT, "Download").click()

        time.sleep(timer) # Importante pois às vezes a aba fecha antes de iniciar o download por questões de internet mesmo
        driver.close()

        driver.switch_to.window(original_window)

        
        # Irá gerar o txt com as informações caso dê algum problema na automação
        arquivo_txt(gera_lista_txt(list_info), formatar_string(list_info), uregi, path)
        """
            gera_lista_txt: 
                Retorna a linha da tabela em formato de string (cortando uma parte dos nomes quando tiver muitos pra não ficar confuso.)
            formatar_string:
                Retorna o nome que o arquivo deve ter conforme o padrão
            uregi:
                O nome da Uregi escolhida que será o nome do arquivo txt
        """


        # Renomeia e move o arquivo pra sua pasta correta
        renomear_e_mover_arquivo(formatar_string(list_info), pegar_nome_original(timer, directory), list_info[2], path, uregi, diretório_de_destino)
        """
            formatar_string:
                Retorna o nome que o arquivo deve ter conforme o padrão
            pegar_nome_original:
                Retorna o endereço e nome do arquivo mais recente criado (ou seja, o arquivo das metas baixado)
            list_info[2]:
                É apenas a SRE da cidade da linha analisada
        """


        # Eu peço pra que a tela desça o tamanho da altura da linha analizada para abrir caminho para que a próxima linha possa estar visível
        scroll += driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]").rect['height']
        driver.execute_script(f"window.scrollTo(500,{scroll})")
        

## Código completo

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from pathlib import Path
import time
import os
import shutil


"""Variáveis que podem/devem ser alteradas conforme a necessidade"""

uregi = "Regional de Alfenas"
# Coloca-se o texto que está na caixa de seleção para escolher a uregi de interesse

cidade_específica = "nenhuma"
#Caso queira pegar os arquivos de uma cidade específica de uma Uregi, "nenhuma" é o padrão pra pegar todas da Uregi

visita_desejada = "todas" 
#As opções são: "todas"; "" pra pegar só as que não colocaram o núm da visita; "1 a 6" o número da visita desejado

linha = 2
# É a linha onde o programa vai iniciar o processo (linha = 2 é o padrão, a primeira linha)

timer = 2
# É o tempo de espera (seg) para fazer o download do arquivo e pegar o nome desse arquivo

directory = Path(r'C:\Users\11020\Downloads') 
# Endereço da pasta onde os arquivos são baixados versão módulo 'os'
path = 'C:\\Users\\11020\\Downloads\\' 
# Endereço da pasta onde os arquivos são baixados versão string
# Lembrar de sempre alterar o caminho do directory e path juntos.

linha_limite = 0
# Para quando se deseja que o programa pare de executar em determinada linha (se não for necessário, esta deve ser 0)

meta_analisada = "Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado"
#É o filtro para a meta específica que se deseja buscar pelo programa, é o valor das células da coluna 'Meta'
"""
Opções:
    Projeto 1 - Meta 1.1 - Mapa da Oferta de Alimentos da AF
    Projeto 2 - Meta 2.1 - Reunião de Mobilização e Divulgação
    Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado
    Projeto 3 - Meta 3.1 - ATER em Agroindústria
    Projeto 4 - Meta 4.1 - Projeto Técnico em Hortas e Pomares
"""

usuario = "61509"
senha = "102492"
#Dados para entrada na intranet

diretório_de_destino = path + "Meta 2.3 - Ater OeM\\" + uregi[12:]
#É a pasta para onde vão os arquivos baixados e renomeados
"""
Outras opções de pastas de destino:
    diretório_de_destino = path + "Evidencias das Uregis - Transição\\" + uregi[12:] + "\\" + meta
"""


# ---------------------------------------------------------------------------------------------


# Inciando a seção e abrindo a tabela da uregi escolhida
driver = webdriver.Chrome()
driver.maximize_window()
original_window = driver.current_window_handle


#Fazendo login na Emater
driver.get("https://www.emater.mg.gov.br/portal.do?flagweb=intranet_pgn_homepage")

driver.find_element(By.XPATH, f"//*[@id='cliente_usuario_login']").send_keys(usuario)
driver.find_element(By.XPATH, f"//*[@id='loginForm']/p[2]/table/tbody/tr[2]/td[2]/input[1]").send_keys(senha)
driver.find_element(By.XPATH, f"//*[@id='enviar']").click()


# Entrando na intranet onde tem os registros das assistências
driver.get("https://www.emater.mg.gov.br/portal.do?flagweb=novaintranet_pgn_pmater_educ_lista_uregi")
time.sleep(timer)

select_element = driver.find_element(By.ID,'mun')
select_object = Select(select_element)

if(cidade_específica == "nenhuma"):
    select_object.select_by_visible_text(uregi)
else:
    select_object.select_by_visible_text(cidade_específica)

search_button = driver.find_element(By.ID, "ok")
search_button.click()

# ---------------------------------------------------------------------------------------------

list_info = []
# Lista que conterá todas as informações de cada linha da tabela

scroll = driver.find_element(By.XPATH, f"/html/body/div[1]/div/div").rect['height'] 
# Armazena a soma das alturas das linhas da tabela

# ---------------------------------------------------------------------------------------------

for i in range(2,linha): #Nesse caso, desço até o fim da linha anterior abrindo espaço pra linha de interesse
    scroll += driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{i}]").rect['height']

driver.execute_script(f"window.scrollTo(500,{scroll})")

"""
# Procura se a linha escolhida está visível, senão o programa irá descer a tela até ela se tornar.
try:
    driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]/td[8]/a/img").click()
except:
    for i in range(2,linha): #Nesse caso, desço até o fim da linha anterior abrindo espaço pra linha de interesse
        scroll += driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{i}]").rect['height']
    driver.execute_script(f"window.scrollTo(500,{scroll})")
else:
    driver.switch_to.window(driver.window_handles[1])
    driver.close()
    driver.switch_to.window(original_window)


finally:
"""    

while True: #Se um dia precisar executar o try-except-else-finally, dê um Tab em tudo daqui pra baixo

    #Parando de pegar assim que ele encontra uma data na coluna de 'Enviado SEI'
    fim_dos_novos = driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]/td[11]").text
    if(len(fim_dos_novos) == 10):
        raise Exception(f"O programa terminou de pegar todos os itens novos")


    #Pegando as informações de cada linha e colocando em uma lista
    list_info.append(f"Índice: {linha - 1}")
    for i in range(2,9):
        list_info.append(driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]/td[{i}]").text)

        
    if(visita_desejada == "todas"):
        #Para pegar uma meta específica (caso não precise, deixe o if e else comentados e dê um Shift Tab em tudo que está dentro desse if)
        if(list_info[3] == meta_analisada):
            executa_ação(linha, uregi, timer, directory, path, meta_analisada, diretório_de_destino, linha_limite, original_window, driver, list_info)
            
            if(linha >= linha_limite and linha_limite != 0):
                raise Exception(f"O programa chegou na linha limite {linha_limite}")
            linha = linha + 1
            list_info.clear() #limpa a lista para o próximo turno 
        else:
            scroll += driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]").rect['height']
            driver.execute_script(f"window.scrollTo(500,{scroll})") #x,y
            
            if(linha >= linha_limite and linha_limite != 0):
                raise Exception(f"O programa chegou na linha limite {linha_limite}")
            linha = linha + 1
            list_info.clear()

            
    else:
        if(list_info[3] == meta_analisada and list_info[4] == visita_desejada):
            executa_ação(linha, uregi, timer, directory, path, meta_analisada, diretório_de_destino, linha_limite, original_window, driver, list_info)
            
            if(linha >= linha_limite and linha_limite != 0):
                raise Exception(f"O programa chegou na linha limite {linha_limite}")
            linha = linha + 1
            list_info.clear() #limpa a lista para o próximo turno 
        else:
            scroll += driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]").rect['height']
            driver.execute_script(f"window.scrollTo(500,{scroll})") #x,y
            
            if(linha >= linha_limite and linha_limite != 0):
                raise Exception(f"O programa chegou na linha limite {linha_limite}")
            linha = linha + 1
            list_info.clear()
    